# Module 3: Data Transformations with Snowpark

## Start a Snowpark session with `get_active_session()`:

In [ ]:
#from snowflake.snowpark.functions import col, max as sp_max, year, month


# We can also use Snowpark for our analyses!
#from snowflake.snowpark.context import get_active_session
#session = get_active_session()


In [4]:
from __future__ import annotations # is a feature that allows for postponed evaluation of type annotations. 
#This means that the annotations won't be evaluated at the time of function or class definition but rather when they're actually needed, such as at runtime or when using tools that analyze type hints.
from snowflake.snowpark.session import Session 
from snowflake.snowpark import DataFrame 
from snowflake.snowpark.functions import col, max as sp_max, year, month
import json

In [5]:
session = Session.builder.configs(json.load(open("/Users/pl157r/.snowsql/connection.json"))).create()

## Load the daily_weather_v using session.table

In [6]:
daily_weather = session.table("tasty_bytes.harmonized.daily_weather_v")

filtered_weather = daily_weather.filter(
    (col("country_desc") == "Germany") &
    (col("city_name") == "Hamburg") &
    (year(col("date_valid_std")) == 2022) &
    (month(col("date_valid_std")) == 2)
)

## Perform filtering, aggregations, and sorting

In [7]:
aggregated_weather = filtered_weather.groupBy(
    "country_desc", "city_name", "date_valid_std"
).agg(
    sp_max("max_wind_speed_100m_mph").alias("max_wind_speed_100m_mph")
)

sorted_weather = aggregated_weather.sort(col("date_valid_std").desc())


## Diplay the results

In [8]:
sorted_weather.show(30)

-------------------------------------------------------------------------------
|"COUNTRY_DESC"  |"CITY_NAME"  |"DATE_VALID_STD"  |"MAX_WIND_SPEED_100M_MPH"  |
-------------------------------------------------------------------------------
|Germany         |Hamburg      |2022-02-28        |23.0                       |
|Germany         |Hamburg      |2022-02-27        |17.0                       |
|Germany         |Hamburg      |2022-02-26        |21.2                       |
|Germany         |Hamburg      |2022-02-25        |34.7                       |
|Germany         |Hamburg      |2022-02-24        |38.2                       |
|Germany         |Hamburg      |2022-02-23        |31.6                       |
|Germany         |Hamburg      |2022-02-22        |37.0                       |
|Germany         |Hamburg      |2022-02-21        |47.7                       |
|Germany         |Hamburg      |2022-02-20        |47.7                       |
|Germany         |Hamburg      |2022-02-

In [9]:
sorted_weather.create_or_replace_view("tasty_bytes.harmonized.windspeed_hamburg_snowpark")

[Row(status='View WINDSPEED_HAMBURG_SNOWPARK successfully created.')]

In [10]:
session.close()